In [1]:
from main import load_corpus,build_chunks

In [6]:
corpus=load_corpus(corpus_dir="docs")
chunked_corpus=build_chunks(corpus,chunk_size=500,overlap=50)

In [8]:
chunked_corpus[:2]

[{'doc_id': 'Corpus.txt',
  'chunk_id': 'Corpus.txt_chunk_0',
  'text': "How Y Combinator Started Y Combinator's 7th birthday was March 11. As usual we were so busy we didn't notice till a few days after. I don't think we've ever managed to remember our birthday on our birthday. On March 11 2005, Jessica and I were walking home from dinner in Harvard Square. Jessica was working at an investment bank at the time, but she didn't like it much, so she had interviewed for a job as director of marketing at a Boston VC fund. The VC fund was doing what now seems a comically familiar thing for a VC fund to do: taking a long time to make up their mind. Meanwhile I had been telling Jessica all the things they should change about the VC business — essentially the ideas now underlying Y Combinator: investors should be making more, smaller investments, they should be funding hackers instead of suits, they should be willing to fund younger founders, etc. At the time I had been thinking about doing so

In [9]:
import os, glob, json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from main import load_corpus, build_chunks

In [10]:
corpus = load_corpus(corpus_dir="docs")
chunked_corpus = build_chunks(corpus, chunk_size=500, overlap=50)
print(f"Total chunks created: {len(chunked_corpus)}")

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode([chunk['text'] for chunk in chunked_corpus], show_progress_bar=True, convert_to_numpy=True)    

Total chunks created: 4


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

d:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
embeddings

array([[ 0.00262532, -0.06060986,  0.0479631 , ..., -0.10234576,
        -0.06416609, -0.02177045],
       [-0.06242754,  0.07373546, -0.01731618, ..., -0.12356471,
        -0.07399752, -0.01288152],
       [-0.02590331, -0.03875893,  0.05224477, ..., -0.00870544,
        -0.07217222, -0.03136087],
       [ 0.0876332 , -0.0484447 ,  0.09743941, ...,  0.03077296,
        -0.02612682, -0.02646555]], dtype=float32)

In [12]:
def normalize(vec):
    norm = np.linalg.norm(vec, axis=1, keepdims=True)
    return vec / norm

vec = normalize(embeddings).astype('float32')
dimension = vec.shape[1]
print(f"Dimension of embeddings: {dimension}")

Dimension of embeddings: 384


In [13]:
index = faiss.IndexFlatIP(dimension)
index.add(vec)
print(f"Total vectors in index: {index.ntotal}")

Total vectors in index: 4


In [14]:
query = "When is the YCOmbinators birthday?"
query_embedding = model.encode([query], convert_to_numpy=True)

In [15]:
query_vec = normalize(query_embedding).astype('float32')

In [16]:
# Retrieve from the index
index.search(query_vec, 2)

(array([[0.35642388, 0.29097164]], dtype=float32), array([[0, 2]]))